# Analyzing data with Tensorflow Data Validation

In [5]:
import numpy as np
import pandas as pd
import tensorflow_data_validation as tfdv


In [10]:
soil_type = [
"1", "2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "2703", "Vanet - Ratake families complex, very stony.",
"3", "2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "3501", "Gothic family.",
"8", "3502", "Supervisor - Limber families complex.",
"9", "4201", "Troutville family, very stony.",
"10", "4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "4744", "Legault family - Rock land complex, stony.",
"13", "4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "5101", "Pachic Argiborolis - Aquolis complex.",
"15", "5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "6101", "Cryaquolis - Cryoborolis complex.",
"17", "6102", "Gateview family - Cryaquolis complex.",
"18", "6731", "Rogert family, very stony.",
"19", "7101", "Typic Cryaquolis - Borohemists complex.",
"20", "7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "7700", "Leighcan family, extremely stony.",
"25", "7701", "Leighcan family, warm, extremely stony.",
"26", "7702", "Granile - Catamount families complex, very stony.",
"27", "7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "7745", "Como - Legault families complex, extremely stony.",
"30", "7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

In [6]:
TRAINING_DATASET='gs://workshop-datasets/covertype/training/covertype_training.csv'
TRAINING_DATASET_WITH_ANOMALIES='gs://workshop-datasets/covertype/training/covertype_training_anomalies.csv'
EVALUATION_DATASET='gs://workshop-datasets/covertype/evaluation/covertype_evaluation.csv'
SERVING_DATASET='gs://workshop-datasets/covertype/serving/covertype_serving.csv'


In [7]:
train_stats = tfdv.generate_statistics_from_csv(
    data_location=TRAINING_DATASET
)

tfdv.visualize_statistics(train_stats)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [8]:
schema = tfdv.infer_schema(train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [11]:
from tensorflow_metadata.proto.v0 import schema_pb2

tfdv.get_feature(schema, 'Soil_Type').type = schema_pb2.FeatureType.BYTES
tfdv.set_domain(schema, 'Soil_Type', schema_pb2.StringDomain(name='Soil_Type', value=soil_type[1::3]))

tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=1, max=7, is_categorical=True))

In [12]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,"[1,7]"
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Soil_Type',"'2702', '2703', '2704', '2705', '2706', '2717', '3501', '3502', '4201', '4703', '4704', '4744', '4758', '5101', '5151', '6101', '6102', '6731', '7101', '7102', '7103', '7201', '7202', '7700', '7701', '7702', '7709', '7710', '7745', '7746', '7755', '7756', '7757', '7790', '8703', '8707', '8708', '8771', '8772', '8776'"


In [13]:
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

train_stats = tfdv.generate_statistics_from_csv(
    data_location=TRAINING_DATASET,
    stats_options=stats_options
)

tfdv.visualize_statistics(train_stats)

In [ ]:
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

train_stats = tfdv.generate_statistics_from_csv(
    data_location=EVALUATION_DATASET,
    stats_options=stats_options
)

tfdv.visualize_statistics(train_stats)

In [ ]:
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

train_stats = tfdv.generate_statistics_from_csv(
    data_location=SERVING_DATASET,
    stats_options=stats_options
)

tfdv.visualize_statistics(train_stats)

In [ ]:
#schema = schema_pb2.Schema()

#schema.feature.add(name='Elevation', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Aspect', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Slope', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Horizontal_Distance_To_Hydrology', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Vertical_Distance_To_Hydrology', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Horizontal_Distance_To_Roadways', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Hillshade_9am', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Hillshade_Noon', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Hillshade_3pm', type=schema_pb2.FeatureType.FLOAT)
#schema.feature.add(name='Horizontal_Distance_To_Fire_Points', type=schema_pb2.FeatureType.FLOAT)

#schema.feature.add(name='Wilderness_Area', type=schema_pb2.FeatureType.BYTES)
#schema.feature.add(name='Soil_Type', type=schema_pb2.FeatureType.BYTES)

#schema.feature.add(name='Cover_Type', type=schema_pb2.FeatureType.INT)
#tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(min=1, max=7, is_categorical=True))